In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [2]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [3]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [4]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [5]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [6]:
df = tripdata[selected_features]

In [7]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 5% NA values

In [8]:
## get the sample size such as 5%, 10%,20% etc
def getSampleSize(df, perc=1):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def induceMissingValues(X_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def induceMissingValuesForDrop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [9]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [10]:
fill_dict ={'start_lat': 40.79865054, 'start_lon': -74.2404433, 'end_lat': 40.79165072 , 'end_lon':-74.28593088,'usertype':'agent',
            'hour': 24, 'temp': 38.6, 'feelslike':49.4,'dew':-25.7,'snowdepth':5.10,'winddir':380.0,'sealevelpressure':819.3,
            'visibility':7.2,'solarradiation': 1109,'conditions': 'muddy','dist': 5.345,'birthyear': 2003,
            'holiday':'unknown','day': 'special','month': 'unknown','gender': 'nonbinary'
        }

In [11]:
def ampute_each_variables(variable, method, X_train, y_train, X_test, y_test):
    if method == "remove":
        X_train , y_train = induceMissingValuesForDrop(X_train,y_train,variable,perc=40)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = induceMissingValuesForDrop(X_test,y_test,variable,perc=40)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train  = induceMissingValues(X_train,variable,perc=40)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test  = induceMissingValues(X_test,variable,perc=40)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
    if method == "bad":
        print("Simulate Imputation for BAD")
        X_train.fillna(fill_dict[variable], inplace=True)
        print('Train  after imputation', X_train.shape, y_train.shape)
        X_test.fillna(fill_dict[variable], inplace=True)
        print('Test  after imputation', X_test.shape, y_test.shape)
        
            
    return X_train , y_train, X_test , y_test

In [12]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [13]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [14]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [14]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())   
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  100938
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  25235
X_test Amputed value count of varaible  25235
Simulate Imputation for NA
Train after NA value removal Imputation (151408, 21) (151408,)
Test after NA value removal Imputation (37852, 21) (37852,)
started bootstrap run is  2023-03-19 23:40:42.503449
stopped bootstrap run is  2023-03-19 23:41:41.672411
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  100938
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21)

Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  100938
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  25235
X_test Amputed value count of varaible  25235
Simulate Imputation for NA
Train after NA value removal Imputation (151408, 21) (151408,)
Test after NA value removal Imputation (37852, 21) (37852,)
started bootstrap run is  2023-03-20 00:00:18.732267
stopped bootstrap run is  2023-03-20 00:01:38.581112
Now running model is .... solarradiation
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  100938
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  25235
X_test Amputed value count of varaible  25235
Simulate

In [15]:
bootstrap_data_drop

{'start_lat': {'R2': [0.5191651452597371, 0.540748583795083],
  'RMSE': [2.200665258700944, 2.2612776931669094],
  'intercept': [-751.5097479527446, -345.41852393267527]},
 'start_lon': {'R2': [0.5191651452597371, 0.540748583795083],
  'RMSE': [2.200665258700944, 2.2612776931669094],
  'intercept': [-751.5097479527446, -345.41852393267527]},
 'end_lat': {'R2': [0.5191651452597371, 0.540748583795083],
  'RMSE': [2.200665258700944, 2.2612776931669094],
  'intercept': [-751.5097479527446, -345.41852393267527]},
 'end_lon': {'R2': [0.5191651452597371, 0.540748583795083],
  'RMSE': [2.200665258700944, 2.2612776931669094],
  'intercept': [-751.5097479527446, -345.41852393267527]},
 'usertype': {'R2': [0.5191651452597371, 0.540748583795083],
  'RMSE': [2.200665258700944, 2.2612776931669094],
  'intercept': [-751.5097479527446, -345.41852393267527]},
 'hour': {'R2': [0.5191651452597371, 0.540748583795083],
  'RMSE': [2.200665258700944, 2.2612776931669094],
  'intercept': [-751.5097479527446, -

### Mode Replacement

In [16]:
bootstrap_data_mode ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-20 00:12:52.042827
stopped bootstrap run is  2023-03-20 00:15:20.418758
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (

Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-20 00:45:19.252133
stopped bootstrap run is  2023-03-20 00:47:23.268873
Now running model is .... dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-20 00:47:37.837954
stopped bootstrap run is  2023-03-20 00:49:29.215840
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
T

In [17]:
bootstrap_data_mode

{'start_lat': {'R2': [0.5196648774163273, 0.5357984833208391],
  'RMSE': [2.208413337522969, 2.2558148002956178],
  'intercept': [-676.7795821565529, -364.82162611201863]},
 'start_lon': {'R2': [0.5189201778169499, 0.5351881626572129],
  'RMSE': [2.209790754045579, 2.2575289281379516],
  'intercept': [-631.6931640281973, -257.6357762272999]},
 'end_lat': {'R2': [0.5196849332043183, 0.5357466852650804],
  'RMSE': [2.20827956438573, 2.2558361778049036],
  'intercept': [-763.8995207143105, -426.63469519493634]},
 'end_lon': {'R2': [0.5194121619984278, 0.5354454516325025],
  'RMSE': [2.209175914102184, 2.256797701667108],
  'intercept': [-850.4814694871529, -485.37314552533064]},
 'usertype': {'R2': [0.5134289681669981, 0.5295505704210444],
  'RMSE': [2.2243317018818907, 2.2711083921800075],
  'intercept': [-658.9032900301836, -366.6459916775767]},
 'hour': {'R2': [0.5183100450063722, 0.5343549774979759],
  'RMSE': [2.211437580027853, 2.2586741158159036],
  'intercept': [-659.122515456318,

### Mean Replacement

In [18]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

bootstrap_data_mean = {}
for var in variables_for_mean_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-20 00:58:17.042782
stopped bootstrap run is  2023-03-20 01:00:01.415394
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (

In [19]:
bootstrap_data_mean

{'start_lat': {'R2': [0.5196880521988124, 0.5358185212671306],
  'RMSE': [2.2083808841298795, 2.255791900568802],
  'intercept': [-706.4318540945388, -382.0778031754303]},
 'start_lon': {'R2': [0.5189192849063191, 0.5351746800979416],
  'RMSE': [2.2097999588526047, 2.2575408372971446],
  'intercept': [-619.4742275140536, -248.079999850179]},
 'end_lat': {'R2': [0.5197103495813169, 0.5357765345562782],
  'RMSE': [2.2082094531492977, 2.2557418988593216],
  'intercept': [-788.4383876550937, -449.428541393436]},
 'end_lon': {'R2': [0.5194476766916843, 0.5354821448568936],
  'RMSE': [2.209104153427642, 2.256767018283796],
  'intercept': [-869.1313113140741, -505.67573222384806]},
 'hour': {'R2': [0.5189352564431791, 0.5349096483958486],
  'RMSE': [2.210014896113055, 2.2573569713102914],
  'intercept': [-654.2497731712599, -371.9288868917486]},
 'temp': {'R2': [0.5196512314876882, 0.5356953220694741],
  'RMSE': [2.2085783526633804, 2.255881329235591],
  'intercept': [-644.9080580941829, -363

### Sampling Replacement

In [20]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simple Imputation using sampling
started bootstrap run is  2023-03-20 01:22:34.721553
stopped bootstrap run is  2023-03-20 01:24:03.203826
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simple Imputation using sampling
started bootstrap run is  2023-03-20 01:24:15.467855
stopped bootstrap run is  2023-03-20 01:25:45.123359
Now running model is .... end_lat
Train (252346, 21) (252346,)
T

started bootstrap run is  2023-03-20 01:52:27.364076
stopped bootstrap run is  2023-03-20 01:53:55.745082
Now running model is .... day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simple Imputation using sampling
started bootstrap run is  2023-03-20 01:54:07.718418
stopped bootstrap run is  2023-03-20 01:55:38.976680
Now running model is .... month
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simple Imputation using sampling
started bootstrap run is  2023-03-20 01:55:54.655775
stopped bootstrap run

In [21]:
bootstrap_data_sample

{'start_lat': {'R2': [0.5197171347863532, 0.535826007910141],
  'RMSE': [2.20836099824745, 2.2558115533454224],
  'intercept': [-661.6618689651234, -357.74937150651215]},
 'start_lon': {'R2': [0.5188851151831791, 0.5350656213516267],
  'RMSE': [2.210189105913085, 2.2578860448313813],
  'intercept': [-752.2781474962615, -413.6427163042174]},
 'end_lat': {'R2': [0.5197389489527113, 0.5357616142598862],
  'RMSE': [2.2083463765403497, 2.255944233627595],
  'intercept': [-698.8561639465961, -381.5315762681968]},
 'end_lon': {'R2': [0.5191192221136065, 0.5353212393268749],
  'RMSE': [2.20960995253579, 2.257238337011424],
  'intercept': [-665.6849164248572, -340.2882548479504]},
 'usertype': {'R2': [0.5104773660869885, 0.5264244008422001],
  'RMSE': [2.2312489596892977, 2.2775349251987596],
  'intercept': [-664.1761260883841, -371.7034356679361]},
 'hour': {'R2': [0.5182485943989414, 0.5342981663032548],
  'RMSE': [2.2117058464499175, 2.2592846698336393],
  'intercept': [-663.6515694073616, -

### Bad Imputation

In [15]:
bootstrap_data_bad ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"bad",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_bad[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 15:52:51.051936
stopped bootstrap run is  2023-03-21 15:56:24.921098
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,

started bootstrap run is  2023-03-21 17:05:18.467855
stopped bootstrap run is  2023-03-21 17:08:46.911370
Now running model is .... dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 17:09:19.421375
stopped bootstrap run is  2023-03-21 17:13:27.300056
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 100938
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  100938
sample size is = 25235
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  25235
Simulate Imputat

In [16]:
bootstrap_data_bad

{'start_lat': {'R2': [0.5196418511662784, 0.5357586675784587],
  'RMSE': [2.2084480114629894, 2.255920499278335],
  'intercept': [-586.7367432256788, -306.2616177972507]},
 'start_lon': {'R2': [0.5186863374323792, 0.5349202689579853],
  'RMSE': [2.2105901156045915, 2.258331298131079],
  'intercept': [-936.9605961779881, -652.8124950642504]},
 'end_lat': {'R2': [0.519541749406004, 0.5356176183338596],
  'RMSE': [2.20873128952712, 2.2561858798888945],
  'intercept': [-544.5654462633597, -255.86379640147283]},
 'end_lon': {'R2': [0.5187854061625385, 0.5349460434590176],
  'RMSE': [2.210228511044227, 2.2578180090568303],
  'intercept': [-424.40157477211096, -146.25023024450977]},
 'usertype': {'R2': [0.5136009612673011, 0.5296399971317864],
  'RMSE': [2.223922353212445, 2.2706031018980264],
  'intercept': [-659.0619132898998, -367.2924039061767]},
 'hour': {'R2': [0.5181446280251649, 0.5341233902521688],
  'RMSE': [2.2119769126633684, 2.259567428236667],
  'intercept': [-663.7785810377889,